In [1]:
import numpy as np
import cv2
import dlib
import math
import sys
import pickle
import argparse
import os
import skvideo.io
from IPython.display import clear_output

skvideo.__version__


ModuleNotFoundError: No module named 'scipy.ndimage'

In [2]:
def video_info(infilename):
    cap = cv2.VideoCapture(infilename)

    if not cap.isOpened():
        print("could not open :", infilename)
        exit(0)

    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print('length : ', length)
    print('width : ', width)
    print('height : ', height)
    print('fps : ', fps)

In [3]:
video_info("./data/video/s1/bbaf2n.mpg")

length :  75
width :  360
height :  288
fps :  25.0


In [4]:
def get_lip_mask(inputs, outputs, fps=25, codec="MJPG"):
    # Dlib requirements.
    predictor_path = './shape_predictor_68_face_landmarks.dat'
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
#     mouth_destination_path = os.path.dirname(outputs) + '/' + 'mouth'
#     if not os.path.exists(mouth_destination_path):
#         os.makedirs(mouth_destination_path)

    inputparameters = {}
    outputparameters = {}
    reader = skvideo.io.FFmpegReader(inputs,
                    inputdict=inputparameters,
                    outputdict=outputparameters)
    video_shape = reader.getShape()
    (num_frames, h, w, c) = video_shape
    print(num_frames, h, w, c)

    # The required parameters
    activation = []
    max_counter = 150
    total_num_frames = int(video_shape[0])
    num_frames = min(total_num_frames,max_counter)
    counter = 0
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Define the writer
    writer = skvideo.io.FFmpegWriter(outputs)


    # Required parameters for mouth extraction.
    width_crop_max = 0
    height_crop_max = 0
    
    # Loop over all frames.
    for frame in reader.nextFrame():
        print('frame_shape:', frame.shape)

        # Process the video and extract the frames up to a certain number and then stop processing.
        if counter > num_frames:
            break

        # Detection of the frame
        frame.setflags()
        detections = detector(frame, 1)

        # 20 mark for mouth
        marks = np.zeros((2, 20))

        # All unnormalized face features.
        Features_Abnormal = np.zeros((190, 1))

        # If the face is detected.
        print(len(detections))
        if len(detections) > 0:
            for k, d in enumerate(detections):

                # Shape of the face.
                shape = predictor(frame, d)

                co = 0
                # Specific for the mouth.
                for ii in range(48, 68):
                    """
                    This for loop is going over all mouth-related features.
                    X and Y coordinates are extracted and stored separately.
                    """
                    X = shape.part(ii)
                    A = (X.x, X.y)
                    marks[0, co] = X.x
                    marks[1, co] = X.y
                    co += 1

                # Get the extreme points(top-left & bottom-right)
                X_left, Y_left, X_right, Y_right = [int(np.amin(marks, axis=1)[0]), int(np.amin(marks, axis=1)[1]),
                                                    int(np.amax(marks, axis=1)[0]),
                                                    int(np.amax(marks, axis=1)[1])]

                # Find the center of the mouth.
                X_center = (X_left + X_right) / 2.0
                Y_center = (Y_left + Y_right) / 2.0

                # Make a boarder for cropping.
                border = 15
                X_left_new = X_left - border
                Y_left_new = Y_left - border
                X_right_new = X_right + border
                Y_right_new = Y_right + border

                # Width and height for cropping(before and after considering the border).
                width_new = X_right_new - X_left_new
                height_new = Y_right_new - Y_left_new
                width_current = X_right - X_left
                height_current = Y_right - Y_left

                # Determine the cropping rectangle dimensions(the main purpose is to have a fixed area).
                if width_crop_max == 0 and height_crop_max == 0:
                    width_crop_max = width_new
                    height_crop_max = height_new
                else:
                    width_crop_max += 1.5 * np.maximum(width_current - width_crop_max, 0)
                    height_crop_max += 1.5 * np.maximum(height_current - height_crop_max, 0)

                # # # Uncomment if the lip area is desired to be rectangular # # # #
                #########################################################
                # Find the cropping points(top-left and bottom-right).
                X_left_crop = int(X_center - width_crop_max / 2.0)
                X_right_crop = int(X_center + width_crop_max / 2.0)
                Y_left_crop = int(Y_center - height_crop_max / 2.0)
                Y_right_crop = int(Y_center + height_crop_max / 2.0)
                #########################################################

                # # # # # Uncomment if the lip area is desired to be rectangular # # # #
                # #######################################
                # # Use this part if the cropped area should look like a square.
                # crop_length_max = max(width_crop_max, height_crop_max) / 2
                #
                # # Find the cropping points(top-left and bottom-right).
                # X_left_crop = int(X_center - crop_length_max)
                # X_right_crop = int(X_center + crop_length_max)
                # Y_left_crop = int(Y_center - crop_length_max)
                # Y_right_crop = int(Y_center + crop_length_max)
                #########################################

                if X_left_crop >= 0 and Y_left_crop >= 0 and X_right_crop < w and Y_right_crop < h:
                    mouth = frame[Y_left_crop:Y_right_crop, X_left_crop:X_right_crop, :]

                    # Save the mouth area.
#                     mouth_gray = cv2.cvtColor(mouth, cv2.COLOR_RGB2GRAY)
#                     cv2.imwrite(mouth_destination_path + '/' + 'frame' + '_' + str(counter) + '.png', mouth_gray)

                    print("The cropped mouth is detected ...")
                    activation.append(1)
                else:
                    cv2.putText(frame, 'The full mouth is not detectable. ', (30, 30), font, 1, (0, 255, 255), 2)
                    print("The full mouth is not detectable. ...")
                    activation.append(0)

        else:
            cv2.putText(frame, 'Mouth is not detectable. ', (30, 30), font, 1, (0, 0, 255), 2)
            print("Mouth is not detectable. ...")
            activation.append(0)


        if activation[counter] == 1:
            # Demonstration of face.
            cv2.rectangle(frame, (X_left_crop, Y_left_crop), (X_right_crop, Y_right_crop), (0, 0, 0), -1)

        # cv2.imshow('frame', frame)
        print('frame number %d of %d' % (counter, num_frames))

        # write the output frame to file
        print("writing frame %d with activation %d" % (counter + 1, activation[counter]))
        writer.writeFrame(frame)
        counter += 1

    writer.close()
    
    the_filename = os.path.dirname(outputs) + '/' + 'activation'
    my_list = activation
    with open(the_filename, 'wb') as f:
        pickle.dump(my_list, f)

In [5]:
def multi_processing(dir_path):
#     for subdir, dirs, files in os.walk(dir_path):
    for file in os.listdir(dir_path):
        input_path = dir_path + os.sep + file
        if input_path.endswith("db"):
            continue
        print(input_path)

        data_path = dir_path.replace("video", "mask")
        if not os.path.isdir(data_path):
            os.makedirs(data_path)
        output_path = data_path + os.sep + file.replace("mpg", "mp4")
        print(output_path)
        get_lip_mask(input_path, output_path)
        clear_output(wait=True)

In [6]:
root_dir = "./data/video"
mp_id = []

for subdir, dirs, files in os.walk(root_dir):
    for dirt in dirs:
        dir_path = subdir + os.sep + dirt
        mp_id.append(dir_path)
        
print(mp_id)
        
#         multi_processing()
            

['./data/video/s25', './data/video/s1', './data/video/s10', './data/video/s11', './data/video/s12', './data/video/s13', './data/video/s14', './data/video/s15', './data/video/s16', './data/video/s17', './data/video/s18', './data/video/s19', './data/video/s2', './data/video/s20', './data/video/s21', './data/video/s22', './data/video/s23', './data/video/s24', './data/video/s26', './data/video/s27', './data/video/s28', './data/video/s29', './data/video/s3', './data/video/s30', './data/video/s31', './data/video/s32', './data/video/s33', './data/video/s4', './data/video/s5', './data/video/s6', './data/video/s7', './data/video/s8', './data/video/s9']


In [ ]:
import multiprocessing
import time
 
#시작시간
start_time = time.time()
 
#멀티 쓰레딩 Pool 사용
pool = multiprocessing.Pool(processes=16) # 현재 시스템에서 사용 할 프로세스 개수
pool.map(multi_processing, mp_id)
pool.close()
pool.join()
 
print("--- %s seconds ---" % (time.time() - start_time))

./data/video/s11/sbwnzn.mpg
The cropped mouth is detected ...
frame_shape: (288, 360, 3)
frame number 1 of 75
frame_shape: (288, 360, 3)
writing frame 2 with activation 1
1
1
1
The cropped mouth is detected ...
frame_shape: (288, 360, 3)
The cropped mouth is detected ...
1
frame number 44 of 75
The cropped mouth is detected ...
writing frame 45 with activation 1
frame number 37 of 75
frame_shape: (288, 360, 3)
1
writing frame 38 with activation 1
The cropped mouth is detected ...
frame_shape: (288, 360, 3)
frame number 35 of 75
1
1
writing frame 36 with activation 1
The cropped mouth is detected ...
frame_shape: (288, 360, 3)
1
The cropped mouth is detected ...
frame number 26 of 75
The cropped mouth is detected ...
frame number 4 of 75
writing frame 27 with activation 1
writing frame 5 with activation 1
frame_shape: (288, 360, 3)
frame_shape: (288, 360, 3)
1
The cropped mouth is detected ...
frame number 2 of 75
writing frame 3 with activation 1
frame_shape: (288, 360, 3)
1
1
The crop

1
frame_shape: (288, 360, 3)
1
writing frame 64 with activation 1
1
frame_shape: (288, 360, 3)
The cropped mouth is detected ...
frame number 26 of 75
frame_shape: (288, 360, 3)
The cropped mouth is detected ...
The cropped mouth is detected ...
frame number 32 of 75
frame number 41 of 75
writing frame 33 with activation 1
writing frame 42 with activation 1
1
frame_shape: (288, 360, 3)
frame_shape: (288, 360, 3)
The cropped mouth is detected ...
writing frame 27 with activation 1
frame number 41 of 75
frame_shape: (288, 360, 3)
1
writing frame 42 with activation 1
The cropped mouth is detected ...
frame number 8 of 75
writing frame 9 with activation 1
1
frame_shape: (288, 360, 3)
frame_shape: (288, 360, 3)
The cropped mouth is detected ...
frame number 56 of 75
writing frame 57 with activation 1
1
The cropped mouth is detected ...
frame_shape: (288, 360, 3)
frame number 44 of 75
writing frame 45 with activation 1
frame_shape: (288, 360, 3)
1
The cropped mouth is detected ...
frame numb

frame_shape: (288, 360, 3)
frame_shape: (288, 360, 3)
frame number 14 of 75
1
writing frame 15 with activation 1
1
frame_shape: (288, 360, 3)
The cropped mouth is detected ...
frame number 1 of 75
The cropped mouth is detected ...
frame_shape: (288, 360, 3)
writing frame 2 with activation 1
frame number 48 of 75
1
writing frame 49 with activation 1
The cropped mouth is detected ...
1
1
frame number 49 of 75
frame_shape: (288, 360, 3)
The cropped mouth is detected ...
writing frame 50 with activation 1
The cropped mouth is detected ...
frame number 67 of 75
1
1
frame_shape: (288, 360, 3)
1
frame_shape: (288, 360, 3)
writing frame 68 with activation 1
The cropped mouth is detected ...
frame number 36 of 75
frame number 29 of 75
The cropped mouth is detected ...
The cropped mouth is detected ...
frame number 60 of 75
The cropped mouth is detected ...
frame number 16 of 75
writing frame 30 with activation 1
writing frame 61 with activation 1
frame number 58 of 75
writing frame 17 with acti

frame number 20 of 75
writing frame 21 with activation 1
frame_shape: (288, 360, 3)
1
frame_shape: (288, 360, 3)
frame_shape: (288, 360, 3)
1
The cropped mouth is detected ...
frame number 6 of 75
writing frame 7 with activation 1
frame_shape: (288, 360, 3)
1
The cropped mouth is detected ...
frame number 34 of 74
The cropped mouth is detected ...
1
1
frame number 2 of 75
The cropped mouth is detected ...
1
writing frame 35 with activation 1
1
writing frame 3 with activation 1
The cropped mouth is detected ...
The cropped mouth is detected ...
frame number 38 of 75
The cropped mouth is detected ...
frame number 2 of 75
frame_shape: (288, 360, 3)
frame number 55 of 75
writing frame 39 with activation 1
frame number 65 of 75
writing frame 3 with activation 1
writing frame 56 with activation 1
writing frame 66 with activation 1
1
frame_shape: (288, 360, 3)
The cropped mouth is detected ...
frame_shape: (288, 360, 3)
1
frame number 5 of 75
1
frame_shape: (288, 360, 3)
frame_shape: (288, 36